In [43]:
from sklearn import svm
import numpy as np
import os
import numpy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
def SVM():
    file_path = os.path.join("ChannelsExtracted","s01", 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',')
    file_path = os.path.join("ChannelsExtracted","s01", 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',')
    BandsFile = np.genfromtxt('training/s01train.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
# SVM Classifier
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
#     print(cm)
#     print("Accuracy score of Valence ")
#     print(accuracy_score(y_test, y_predict)*100)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    	
# SVM Classifier
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    cm = confusion_matrix(z_test, z_predict)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100]
#     print(cm)
#     print("Accuracy score of Arousal ")
#     print(accuracy_score(z_test, z_predict)*100)

# print("Valence test")
# for i in range (len(y_test)):
#     print(y_test[i])
# print("Valence Train")
# for i in range (len(y_train)):
#     print(y_train[i])
# print("Valence predict")
# for i in range (len(y_predict)):
#     print(y_predict[i])
    
# print("$$$$$$$Arousal$$$$$$$")


def Run1000():
    arousal=[]
    valence=[]
    for i in range (100):
        svmpca=SVM()
        arousal+=[svmpca[0]]
        valence+=[svmpca[1]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))

if __name__ == "__main__":
        Run1000()

Arousal 56.125
Counter({50.0: 35, 62.5: 23, 75.0: 22, 37.5: 12, 25.0: 6, 87.5: 2})
Valence 35.5
Counter({37.5: 35, 25.0: 26, 50.0: 17, 12.5: 13, 62.5: 8, 75.0: 1})
